In [23]:
import pathlib
from PIL import Image
import cv2
import os
import numpy as np

import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

height, width = 2799, 2812
rowHeight = int(height / 15) - 1
colWidth = int(width / 15)
print(rowHeight, colWidth)

outputfilePath = pathlib.Path("dataTestProcessed")

img_height = 184
img_width = 187
class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y',
               'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

185 187


# Load Model

In [24]:
TF_MODEL_FILE_PATH = "imageClassificationModel.tflite"
interpreter = tf.lite.Interpreter(model_path=TF_MODEL_FILE_PATH)
print(interpreter.get_signature_list())

classify_lite = interpreter.get_signature_runner('serving_default')

{'serving_default': {'inputs': ['rescaleImage_input'], 'outputs': ['output']}}


# Generate character matrix

In [37]:
def generateCharacterMatrix(pageNumber):
    print("Generating raw matrix")
    charMatrix = []
    x, y = 0, 0
    while x < 15:
        charMatrixRow = []
        y = 0
        while y < 15:
            imageName = "{}_{}_{}.jpg".format(pageNumber, x, y)
            imagePath = outputfilePath.joinpath(imageName)
            img = tf.keras.utils.load_img(str(imagePath), target_size=(img_height, img_width) )
            img_array = tf.keras.utils.img_to_array(img)
            img_array = tf.expand_dims(img_array, 0)

            predictions_lite = classify_lite(rescaleImage_input=img_array)['output']
            score_lite = tf.nn.softmax(predictions_lite)

            # print("Score: {} | Position: {} | Letter: {}".format(np.max(score_lite), np.argmax(score_lite), class_names[np.argmax(score_lite)]))
            charMatrixRow.append(class_names[np.argmax(score_lite)].upper())
            y += 1
        x += 1
        charMatrix.append(charMatrixRow)
    return np.array(charMatrix)

# Find Santa Routine

In [139]:
santas = ["ARUSHI", "CHANDAN", "MEGHANA", "SHRIDAR", "JERRY", "VAISHNAVI", "UTKARSH", "BRUNDA", "MOHAN", "SANDIL", "PARVATHY", "SIVA", "JUGNU", "HAMSINI", "ABUZAR", "KASHMEERA", "ABHIJEET", "DHEEMANTH", "NEELKANT", "ANURAAG", "NIRAV", "DEEPAK", "KAVI", "SNEHAL", "KARTHIK", "GAURAV", "NAVANEETA", "VIGNESH", "DEEPAK", "SUSHMITHA", "MOHNEESH", "YASH", "VIKAS", "GOPAKUMAR"]

def searchByRows(charImageMatrix):
	print("Searching rows")
	for i in range(15):
		line =""
		for c in charImageMatrix[i, :]:
			line += c
		for s in santas:
			count = line.count(s)
			if count > 0:
				return i, s
			count = line[::-1].count(s)
			if count > 0:
				return i, s
	return 0, ""

def searchByColumns(charImageMatrix):
	print("Searching columns")
	for i in range(15):
		line =""
		for c in charImageMatrix[:, i]:
			line += c
		for s in santas:
			count = line.count(s)
			if count > 0:
				return i, s
			count = line[::-1].count(s)
			if count > 0:
				return i, s
	return 0, ""

def searchDiagonally(charImageMatrix):
	print("Searching diagonally")
	max = 15
	diagonals = []
	for i in range(15):
		line = ""
		x = 0
		for loop in range(15):
			if x + i < 15:
				line += charImageMatrix[x][x+i]
			x += 1
		for s in santas:
			count = line.count(s)
			if count > 0:
				return x, i, s
			count = line[::-1].count(s)
			if count > 0:
				return x, i, s
	for i in range(15):
		i = i+1
		if(i == 15):
			break
		line = ""
		x = 0
		for loop in range(15):
			if x + i < 15:
				line += charImageMatrix[x+i][x]
			x += 1
		for s in santas:
			count = line.count(s)
			if count > 0:
				return x, i, s
			count = line[::-1].count(s)
			if count > 0:
				return x, i, s
	return 0, 0, ""


# Find Santa

> Start here

In [143]:
def findSanta(pageNumber):
	charImageMatrix = generateCharacterMatrix(pageNumber)
	print(charImageMatrix)
	row, santa = searchByRows(charImageMatrix)
	if santa != "":
		return santa
	col, santa = searchByColumns(charImageMatrix)
	if santa != "":
		return santa
	col, row, santa = searchDiagonally(charImageMatrix)
	if santa != "":
		return santa
	col, row, santa = searchDiagonally(charImageMatrix[::-1])
	return santa

# 1 -> Jerry
# 2 -> Vaishnavi
# 3 -> Kashmeera
# 4 -> Neel

pageNumber = 1
print("-------- [START] - {} --------".format(pageNumber))
print("Santa -> {}".format(findSanta(pageNumber)))
print("-------- [DONE] - {} --------\n".format(pageNumber))

-------- [START] - 1 --------
Generating raw matrix
[['P' 'A' 'H' 'X' 'J' 'K' 'D' 'E' 'P' 'S' 'C' 'F' 'J' 'V' 'F']
 ['T' 'A' 'Z' 'J' 'T' 'R' 'A' 'N' 'V' 'A' 'B' 'A' 'V' 'H' 'J']
 ['O' 'F' 'F' 'E' 'G' 'O' 'G' 'I' 'O' 'Z' 'Q' 'K' 'Y' 'E' 'Y']
 ['A' 'J' 'V' 'Y' 'I' 'X' 'J' 'F' 'C' 'L' 'I' 'F' 'V' 'C' 'B']
 ['H' 'T' 'T' 'L' 'P' 'M' 'V' 'Z' 'I' 'Q' 'E' 'P' 'J' 'Z' 'R']
 ['G' 'R' 'N' 'Z' 'Y' 'Z' 'I' 'S' 'G' 'V' 'W' 'W' 'R' 'Z' 'A']
 ['K' 'B' 'K' 'I' 'K' 'B' 'T' 'H' 'V' 'V' 'N' 'O' 'I' 'J' 'R']
 ['L' 'Z' 'E' 'D' 'K' 'K' 'U' 'C' 'S' 'Z' 'L' 'R' 'V' 'K' 'R']
 ['D' 'Q' 'E' 'E' 'J' 'D' 'D' 'N' 'V' 'U' 'B' 'O' 'F' 'J' 'B']
 ['G' 'W' 'G' 'X' 'A' 'H' 'A' 'A' 'B' 'C' 'R' 'V' 'G' 'W' 'I']
 ['W' 'Z' 'R' 'A' 'H' 'F' 'N' 'G' 'O' 'R' 'P' 'A' 'B' 'P' 'Y']
 ['O' 'L' 'A' 'G' 'B' 'X' 'G' 'H' 'S' 'F' 'G' 'V' 'B' 'B' 'P']
 ['F' 'E' 'K' 'Q' 'W' 'N' 'J' 'I' 'O' 'D' 'R' 'S' 'T' 'I' 'P']
 ['V' 'K' 'S' 'D' 'O' 'W' 'K' 'Q' 'A' 'W' 'B' 'H' 'R' 'J' 'Y']
 ['V' 'C' 'G' 'M' 'X' 'Z' 'C' 'H' 'Y' 'B' 'Z' 'K' 'O' 'I' 'M']]
Se